In [12]:
import requests

# URL for the category data
url = "https://world.openfoodfacts.org/categories.json"

# Make the API request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    try:
        # Attempt to decode JSON
        categories = response.json()
        print(categories)  # Or process the data as needed
    except ValueError:
        # Handle the case where JSON is invalid
        print("Invalid JSON received:")
        print(response.text)
else:
    print(f"Failed to fetch data: Status code {response.status_code}")
    print(response.text)


Invalid JSON received:
<!DOCTYPE html><html><head><meta name="robots" content="noindex"></head><body><h1>NOINDEX</h1><p>We detected that your browser is a web crawling bot, and this page should not be indexed by web crawlers. If this is unexpected, contact us on Slack or write us an email at <a href="mailto:contact@openfoodfacts.org">contact@openfoodfacts.org</a>.</p></body></html>


In [14]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = "https://world.openfoodfacts.org/categories.json"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    try:
        categories = response.json()
        print(categories)
    except ValueError:
        print("Invalid JSON received:")
        print(response.text)
else:
    print(f"Failed to fetch data: Status code {response.status_code}")
    print(response.text)


{'count': 70602, 'tags': [{'id': 'en:plant-based-foods-and-beverages', 'known': 1, 'name': 'Plant-based foods and beverages', 'products': 471597, 'url': 'https://world.openfoodfacts.org/category/plant-based-foods-and-beverages'}, {'id': 'en:plant-based-foods', 'known': 1, 'name': 'Plant-based foods', 'products': 410530, 'url': 'https://world.openfoodfacts.org/category/plant-based-foods'}, {'id': 'en:snacks', 'known': 1, 'name': 'Snacks', 'products': 284269, 'sameAs': ['https://www.wikidata.org/wiki/Q749316'], 'url': 'https://world.openfoodfacts.org/category/snacks'}, {'id': 'en:sweet-snacks', 'known': 1, 'name': 'Sweet snacks', 'products': 209311, 'url': 'https://world.openfoodfacts.org/category/sweet-snacks'}, {'id': 'en:beverages', 'known': 1, 'name': 'Beverages', 'products': 180483, 'sameAs': ['https://www.wikidata.org/wiki/Q40050'], 'url': 'https://world.openfoodfacts.org/category/beverages'}, {'id': 'en:dairies', 'known': 1, 'name': 'Dairies', 'products': 151885, 'sameAs': ['https

In [22]:
import requests
import pandas as pd

def fetch_categories():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    url = "https://world.openfoodfacts.org/categories.json"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('tags', [])
    else:
        print(f"Failed to fetch categories: {response.status_code}")
        return []

def fetch_products_for_category(category_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    url = f"https://world.openfoodfacts.org/category/{category_id}/country/canada.json?page_size=200"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('products', [])
    else:
        print(f"Failed to fetch products for category {category_id}: {response.status_code}")
        return []

def main():
    categories = fetch_categories()
    all_products = []

    for category in categories:
        category_id = category['id']
        category_name = category['name']
        print(f"Fetching products for category: {category_name}")

        products = fetch_products_for_category(category_id)
        for product in products:
            product['category_name'] = category_name  # Add category name to each product
            all_products.append(product)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_products)
    df.to_csv('../data/openfoodfacts_products.csv', index=False)
    print("Data saved to 'openfoodfacts_products.csv'.")

if __name__ == '__main__':
    main()


Fetching products for category: Plant-based foods and beverages
Fetching products for category: Plant-based foods
Fetching products for category: Snacks
Fetching products for category: Sweet snacks
Fetching products for category: Beverages
Fetching products for category: Dairies
Failed to fetch products for category en:dairies: 429
Fetching products for category: Cereals and potatoes
Failed to fetch products for category en:cereals-and-potatoes: 429
Fetching products for category: Meats and their products
Failed to fetch products for category en:meats-and-their-products: 429
Fetching products for category: Fermented foods
Failed to fetch products for category en:fermented-foods: 429
Fetching products for category: Fermented milk products
Failed to fetch products for category en:fermented-milk-products: 429
Fetching products for category: Fruits and vegetables based foods
Failed to fetch products for category en:fruits-and-vegetables-based-foods: 429
Fetching products for category: Cond

In [27]:
import requests
import pandas as pd
imort time

def fetch_products_for_category(category_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    url = f"https://world.openfoodfacts.org/category/{category_id}/country/world.json?page_size=200"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('products', [])
    else:
        print(f"Failed to fetch products for category {category_id}: {response.status_code}")
        return []

def extract_relevant_data(products):
    # Define the fields you want to extract
    fields = ['brands', 'categories', 'countries', 'food_groups_tags', 'food_groups', 'ingredients_text']
    filtered_data = [{field: product.get(field, '') for field in fields} for product in products]
    return filtered_data

def main():
    category_id = 'snacks'  # Example category
    products = fetch_products_for_category(category_id)
    relevant_data = extract_relevant_data(products)

    # Convert the filtered data to a DataFrame and save it to a CSV file
    df = pd.DataFrame(relevant_data)
    df.to_csv('filtered_openfoodfacts_data.csv', index=False)
    print("Data saved to 'filtered_openfoodfacts_data.csv'.")

if __name__ == '__main__':
    main()


Data saved to 'filtered_openfoodfacts_data.csv'.


In [45]:
import requests
import pandas as pd
import time

def fetch_categories():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    url = "https://world.openfoodfacts.org/categories.json"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('tags', [])
    else:
        print(f"Failed to fetch categories: {response.status_code}")
        return []

def fetch_products_for_category(category_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    url = f"https://world.openfoodfacts.org/category/{category_id}/country/canada.json?page_size=200"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('products', [])
    elif response.status_code == 429:
        print(f"Rate limit exceeded for category {category_id}. Retrying in 60 seconds...")
        time.sleep(60)  # Wait for 60 seconds before retrying
        return fetch_products_for_category(category_id)  # Recursive retry
    else:
        print(f"Failed to fetch products for category {category_id}: {response.status_code}")
        return []

def extract_relevant_fields(products):
    fields = ['brands', 'categories', 'countries', 'food_groups_tags', 'food_groups', 'ingredients_text']
    filtered_products = []
    for product in products:
        filtered_product = {field: product.get(field, '') for field in fields}
        filtered_products.append(filtered_product)
    return filtered_products

def main():
    categories = fetch_categories()
    countries = ['canada', 'united states']
    all_products = []

    for category in categories:
        category_id = category['id']
        category_name = category['name']
        print(f"Fetching products for category: {category_name}")

        products = fetch_products_for_category(category_id)
        filtered_products = extract_relevant_fields(products)
        for product in filtered_products:
            product['category_name'] = category_name  # Add category name to each product
            product['country'] = country
            all_products.append(product)
    time.sleep(1)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_products)
    df.to_csv('openfoodfacts_products.csv', index=False)
    print("Data saved to 'openfoodfacts_products.csv'.")

if __name__ == '__main__':
    main()


Fetching products for category: Plant-based foods and beverages
Rate limit exceeded for category en:plant-based-foods-and-beverages. Retrying in 60 seconds...
Fetching products for category: Plant-based foods
Fetching products for category: Snacks
Fetching products for category: Sweet snacks
Fetching products for category: Beverages
Fetching products for category: Dairies
Rate limit exceeded for category en:dairies. Retrying in 60 seconds...
Fetching products for category: Cereals and potatoes
Fetching products for category: Meats and their products
Fetching products for category: Fermented foods
Fetching products for category: Fermented milk products
Fetching products for category: Fruits and vegetables based foods
Fetching products for category: Condiments
Fetching products for category: Meats
Fetching products for category: Desserts
Fetching products for category: Biscuits and cakes
Fetching products for category: Meals
Rate limit exceeded for category en:meals. Retrying in 60 secon

KeyboardInterrupt: 

In [58]:
import requests
import pandas as pd
import time

def fetch_categories():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    url = "https://world.openfoodfacts.org/categories.json"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('tags', [])
    else:
        print(f"Failed to fetch categories: {response.status_code}")
        return []

def fetch_products_for_category(category_id, country):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    url = f"https://world.openfoodfacts.org/category/{category_id}/country/{country}.json?page_size=200"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('products', [])
    elif response.status_code == 429:
        print(f"Rate limit exceeded for category {category_id} in {country}. Retrying in 60 seconds...")
        time.sleep(60)
        return fetch_products_for_category(category_id, country)
    else:
        print(f"Failed to fetch products for category {category_id} in {country}: {response.status_code}")
        return []

def extract_relevant_fields(products):
    fields = ['brands', 'product_name','categories', 'countries', 'food_groups_tags', 'food_groups', 'ingredients_text']
    filtered_products = []
    for product in products:
        filtered_product = {field: product.get(field, '') for field in fields}
        filtered_products.append(filtered_product)
    return filtered_products

def main():
    categories = fetch_categories()
    countries = ['canada', 'united-states','world']  # Now includes both countries
    all_products = []

    for category in categories:
        category_id = category['id']
        category_name = category['name']
        for country in countries:
            print(f"Fetching products for category: {category_name} in {country}")
            products = fetch_products_for_category(category_id, country)
            filtered_products = extract_relevant_fields(products)
            for product in filtered_products:
                product['category_name'] = category_name
                product['country'] = country
                all_products.append(product)
            time.sleep(1)  # Rate limiting by adding a delay

    df = pd.DataFrame(all_products)
    df.to_csv('../data/openfoodfacts_products_world.csv', index=False)
    print("Data saved to 'openfoodfacts_products.csv'.")

if __name__ == '__main__':
    main()


Fetching products for category: Plant-based foods and beverages in canada
Fetching products for category: Plant-based foods and beverages in united-states
Fetching products for category: Plant-based foods and beverages in world
Fetching products for category: Plant-based foods in canada
Fetching products for category: Plant-based foods in united-states
Fetching products for category: Plant-based foods in world
Rate limit exceeded for category en:plant-based-foods in world. Retrying in 60 seconds...
Fetching products for category: Snacks in canada
Fetching products for category: Snacks in united-states
Fetching products for category: Snacks in world
Fetching products for category: Sweet snacks in canada
Fetching products for category: Sweet snacks in united-states
Rate limit exceeded for category en:sweet-snacks in united-states. Retrying in 60 seconds...
Fetching products for category: Sweet snacks in world
Fetching products for category: Beverages in canada
Fetching products for categ